In [448]:
import requests
# pprint is pretty print (formats the JSON)
from pprint import pprint
from IPython.display import HTML
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['text.color'] = 'k'

In [449]:
subscription_key = 'e41c34b3bcf841d0859d6aba9e8e1494'
assert subscription_key
text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/"

language_api_url = text_analytics_base_url + "languages"
print(language_api_url)

sentiment_api_url = text_analytics_base_url + "sentiment"
print(sentiment_api_url)

https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/languages
https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.1/sentiment


In [450]:
tweets = pd.read_csv('data/clean/19VzlaEnLaCalle_2017_04_19_to_2017_04_22_mod.csv', sep=';',error_bad_lines=False)
tweets.head()

b'Skipping line 17709: expected 10 fields, saw 11\nSkipping line 18724: expected 10 fields, saw 11\nSkipping line 24832: expected 10 fields, saw 11\n'


username              date  retweets  favorites  \
0  XIOMARAPINAR2  2017-04-21 19:44         0          0   
1    desdevargas  2017-04-21 17:37         1          1   
2  incesamazonas  2017-04-21 16:38         0          0   
3  incesamazonas  2017-04-21 16:28         0          0   
4   FreeCubaVzla  2017-04-21 14:13         0          0   

                                                text  geo           mentions  \
0  Somos el 95% de OPOSITORES el CNE lo q tiene q...  NaN                NaN   
1  # 19VzlaEnLaCalle y en los puntos rojos de Var...  NaN  @CuatroFDigital @   
2  Retweeted ANTV VENEZUELA (@ANTV_VENEZUELA): # ...  NaN    @ANTV_VENEZUELA   
3  Retweeted Elias Jaua ( @ JauaMiranda): # 19vzl...  NaN                  @   
4  Retweeted Mando Militar ( @ MandoMilitar): # 1...  NaN                  @   

  hashtags            id                                          permalink  
0        #  8.555678e+17  https://twitter.com/XIOMARAPINAR2/status/85556...  
1        #  8.555359e+17  https://twitter.com/desdevargas/status/8555359...  
2      # #  8.555210e+17  https://twitter.com/incesamazonas/status/85552...  
3        #  8.555187e+17  https://twitter.com/incesamazonas/status/85551...  
4        #  8.554845e+17  https://twitter.com/FreeCubaVzla/status/855484...

In [451]:
tweets['text'][2]

'Retweeted ANTV VENEZUELA (@ANTV_VENEZUELA): # 19VzlaEnLaCalle El # 19Abr el Pueblo con paz toma Caracas y... http://fb.me/606ll7gK6'

In [452]:
#create binary indicator of whether or not this is a retweet
tweets['retweet'] = tweets['text'].apply(lambda x: re.sub(r'Retweeted.*\):','',x))


In [453]:
#create a 'clean text' column that has stripped @ and the follwing username, # and 'retweet'
tweets['clean_text'] = tweets['text'].apply(lambda x: re.sub('Retweeted.*\):','',x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x:re.sub('(http.*$)','',x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x:re.sub('(pic.twitter.*$)','',x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x:re.sub('([@#][\w_-]+)','',x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x:re.sub('([@] [^ ]+)','', x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x:re.sub('([#] [^ ]+)','', x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x: re.sub('[^\w]', ' ', x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x: re.sub('(['  '.*])', ' ', x))

In [454]:
tweets = tweets[tweets['clean_text']!=' ']

In [455]:
tweets.head()

username              date  retweets  favorites  \
0  XIOMARAPINAR2  2017-04-21 19:44         0          0   
1    desdevargas  2017-04-21 17:37         1          1   
2  incesamazonas  2017-04-21 16:38         0          0   
3  incesamazonas  2017-04-21 16:28         0          0   
4   FreeCubaVzla  2017-04-21 14:13         0          0   

                                                text  geo           mentions  \
0  Somos el 95% de OPOSITORES el CNE lo q tiene q...  NaN                NaN   
1  # 19VzlaEnLaCalle y en los puntos rojos de Var...  NaN  @CuatroFDigital @   
2  Retweeted ANTV VENEZUELA (@ANTV_VENEZUELA): # ...  NaN    @ANTV_VENEZUELA   
3  Retweeted Elias Jaua ( @ JauaMiranda): # 19vzl...  NaN                  @   
4  Retweeted Mando Militar ( @ MandoMilitar): # 1...  NaN                  @   

  hashtags            id                                          permalink  \
0        #  8.555678e+17  https://twitter.com/XIOMARAPINAR2/status/85556...   
1        #  8.555359e+17  https://twitter.com/desdevargas/status/8555359...   
2      # #  8.555210e+17  https://twitter.com/incesamazonas/status/85552...   
3        #  8.555187e+17  https://twitter.com/incesamazonas/status/85551...   
4        #  8.554845e+17  https://twitter.com/FreeCubaVzla/status/855484...   

                                             retweet  \
0  Somos el 95% de OPOSITORES el CNE lo q tiene q...   
1  # 19VzlaEnLaCalle y en los puntos rojos de Var...   
2   # 19VzlaEnLaCalle El # 19Abr el Pueblo con pa...   
3   # 19vzlaenlacalle logró el triunfo de la Inde...   
4   # 19vzlaenlacalle Momento cuando las personas...   

                                          clean_text  
0  Somos el 95  de OPOSITORES el CNE lo q tiene q...  
1   y en los puntos rojos de Vargas en defensa de...  
2           El  el Pueblo con paz toma Caracas y      
3    logró el triunfo de la Independencia y La Pa...  
4    Momento cuando las personas son obligadas a ...

In [478]:
def language_and_sentiment(df):
    
    #Language
    id_count = 0
    documents = {'documents': []}
    
    #compile language document
    for text in df['clean_text']:        
        documents['documents'].append({'id': '{}'.format(id_count), 'text': text})
        id_count += 1
        
    #feed compiled doc to API    
    headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
    response  = requests.post(language_api_url, headers=headers, json=documents)
    languages = response.json()
    
    new_lang_dict = {}
    for entry in languages['documents']:
        new_lang_dict[entry['id']] = entry['detectedLanguages'][0]['iso6391Name']
    
    
    langs = []
    for i in range(0,len(df)):
        if str(i) in new_lang_dict.keys():
            langs.append(new_lang_dict[str(i)])
        else:
            langs.append(np.nan)
        
    

    #append languages to df
    df['language'] = langs
    
    
    ## Sentiment
    id_count = 0
    documents = {'documents': []}
    
    #compile sentiment document
    for text in df['clean_text']:
        documents['documents'].append({'id': '{}'.format(id_count), 'language': langs[id_count], 'text': text})
        id_count += 1
        
    #feed compiled sentiment doc to API
    headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
    response  = requests.post(sentiment_api_url, headers=headers, json=documents)
    sentiments = response.json()
    
    new_sent_dict = {}
    for entry in sentiments['documents']:
        new_sent_dict[entry['id']] = entry['score']
    
    
    sents = []
   
    for i in range(0,len(df)):
        if str(i) in new_sent_dict.keys():
            sents.append(new_sent_dict[str(i)])
        else:
            sents.append(np.nan)
        
        
    
    df['sentiment'] = sents
    
    return df

        
    



In [482]:
def chunker(seq, size):
    return (seq[pos:pos + size].copy() for pos in range(0, len(seq), size))

chunk_dfs = []
for i in chunker(tweets,500):
    time.sleep(10)
    chunk_dfs.append(language_and_sentiment(i))
    
tweets_with_sent = pd.concat(chunk_dfs)

In [487]:
tweets_with_sent.to_csv('data/sentiment/19VzlaEnLaCalle_2017_04_19_to_2017_04_22_sent.csv')